# Encontrando películas similares

In [ ]:
import pandas as pd

m_cols = ['anime_id', 'name']
animes = pd.read_csv('..\\data\\anime.csv', usecols = m_cols)
r_cols = ['user_id', 'anime_id', 'rating']
ratings = pd.read_csv('..\\data\\rating.csv',  sep=',', usecols=r_cols)

# Filtramos por ratings válidos (mayores a 0)
ratings_filtrados = ratings[ratings['rating'] >= 0]

anime_counts = ratings_filtrados.groupby('anime_id')['rating'].count()

animes_populares = anime_counts[anime_counts > 100].index

# Filtramos por número mínimo de ratings que tenga un anime
ratings = ratings_filtrados[ratings_filtrados['anime_id'].isin(animes_populares)]


# Eliminamos los duplicados para anime y ratings checkeando previamente si hay alguno en cada uno de los csv
duplicates_ratings = ratings.duplicated(subset=['user_id', 'anime_id']).sum() # Porque suma???
print(duplicates_ratings)
if duplicates_ratings > 0: # Mayor que 0???????
    dup_mask = ratings.duplicated(subset=['user_id', 'anime_id'], keep=False)

duplicates_animes = animes.duplicated(subset=['anime_id', 'name']).sum()
if duplicates_animes > 0:
    dup_mask = animes.duplicated(subset=['anime_id', 'name'], keep=False)

# Comprovamos que los rating negativos se hayan eliminado
print("Mínimo rating:", ratings['rating'].min())
ratings.head()
# animes.head()



Mínimo rating: 1


,user_id,anime_id,rating
47,1,8074,10
81,1,11617,10
83,1,11757,10
101,1,15451,10
153,2,11771,10


In [25]:
counts_per_anime = ratings['anime_id'].value_counts()
print(counts_per_anime.describe())

for p in [50, 75, 90, 95, 99]:
    print(f"P{p} =", int(counts_per_anime.quantile(p/100)))

count     4129.000000
mean      1501.150400
std       2545.440184
min        101.000000
25%        230.000000
50%        570.000000
75%       1591.000000
max      34226.000000
Name: count, dtype: float64
P50 = 570
P75 = 1591
P90 = 3782
P95 = 6135
P99 = 12454


In [26]:
counts_per_user = ratings['user_id'].value_counts()
print(counts_per_user.describe())

for p in [50, 75, 90, 95, 99]:
    print(f"P{p} =", int(counts_per_user.quantile(p/100)))

count    69566.000000
mean        89.098841
std        128.510716
min          1.000000
25%         13.000000
50%         44.000000
75%        113.000000
max       2781.000000
Name: count, dtype: float64
P50 = 44
P75 = 113
P90 = 226
P95 = 326
P99 = 616


In [1]:
# Comptem usuaris abans del filtre
users_before = ratings['user_id'].nunique()

# Apliquem filtre d'usuaris
MIN_RATINGS_USER = 5
counts_user = ratings['user_id'].value_counts()
valid_users = counts_user[counts_user >= MIN_RATINGS_USER].index
ratings = ratings[ratings['user_id'].isin(valid_users)].copy()

# Comptem usuaris i files després
users_after = ratings['user_id'].nunique()
rows_after = len(ratings)

print("Usuaris abans del filtre:", users_before)
print("Usuaris després del filtre:", users_after)
print("Files totals després del filtre:", rows_after)

# percetantge de dades que he conservat
percentatge = len(ratings) / len(ratings) * 100
print(f"Percentatge de files conservades: {percentatge:.2f}%")


MAX_RATINGS_USER = counts_user.quantile(0.99)  # ≈ 617
valid_users = counts_user[
    (counts_user >= 5) &
    (counts_user <= MAX_RATINGS_USER)
].index

ratings = ratings[ratings['user_id'].isin(valid_users)].copy()
# Comptem usuaris i files després del filtre superior
users_after2 = ratings['user_id'].nunique()
rows_after2 = len(ratings)

print("Usuaris després del filtre superior:", users_after2)
print("Files totals després del filtre superior:", rows_after2)

# Percentatge de files conservades respecte a l'original
percentatge2 = len(ratings) / len(ratings) * 100
print(f"Percentatge de files conservades: {percentatge2:.2f}%")

perdua_usuaris = users_before - users_after2
print(f"Usuaris eliminats en total: {perdua_usuaris}")

NameError: name 'ratings' is not defined

In [28]:
# ratings = pd.merge(animes, ratings)

animeRatings = ratings.pivot_table(index=['user_id'],columns=['anime_id'],values='rating')
animeRatings.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,33241,33338,33372,33421,33524,33558,33569,33964,34103,34240
user_id,,,,,,,,,,,,,,,,,,,,,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,8.0,NaN,NaN,6.0,NaN,6.0,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Puedo usar las columnas para encontrar las calificaciones de un anime

In [29]:
HatsukoiRatings = animeRatings[7669]
HatsukoiRatings = HatsukoiRatings.dropna()
HatsukoiRatings.head()

user_id
160    7.0
455    9.0
481    9.0
553    9.0
741    6.0
Name: 7669, dtype: float64

"corrwith" hace el cálculo de correlación entre Hatsukoi con todos los otros animes.

In [30]:
similarAnimesToHatsukoi = animeRatings.corrwith(HatsukoiRatings)
similarAnimesToHatsukoi = similarAnimesToHatsukoi.dropna()

df = pd.DataFrame(similarAnimesToHatsukoi)
df.head(10)

C:\Users\dsegu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\dsegu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\dsegu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_function_base_impl.py:3057: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\dsegu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\_function_base_impl.py:2914: RuntimeWarning: divide by zero encountered in divid

,0
anime_id,
1,0.113625
5,0.411978
6,0.208126
7,0.342605
15,0.240057
16,0.134487
17,0.566947
18,0.475909
19,0.163491


In [31]:
similarAnimesToHatsukoi.sort_values(ascending=False).head(40)

anime_id
33032    1.0
33046    1.0
32309    1.0
32365    1.0
6546     1.0
10074    1.0
6877     1.0
7113     1.0
7669     1.0
3670     1.0
8026     1.0
9088     1.0
9232     1.0
2106     1.0
2043     1.0
1942     1.0
2014     1.0
1728     1.0
2153     1.0
10916    1.0
2288     1.0
614      1.0
1351     1.0
1493     1.0
23725    1.0
424      1.0
27967    1.0
27503    1.0
473      1.0
1661     1.0
1448     1.0
1491     1.0
1492     1.0
1484     1.0
1603     1.0
1598     1.0
1635     1.0
127      1.0
516      1.0
23777    1.0
dtype: float64

Hay muchos animes que en realidad tienen poco que ver con Hatsukoi. Probablemente hay un par de personas que vieron Hatsukoi y también Full Speed por ejemplo, que no es un anime muy conocido y eso ya le da un puntaje de similaridad alto. ¿Cómo se puede arreglar?

Agrupamos por título y mostramos la cantidad de calificaciones.

In [32]:
import numpy as np
#animeStats = ratings.groupby('name').agg({'rating': [np.size, np.mean]})
#animeStats.head(15)

animeStatsNuevo = ratings.groupby('anime_id').agg({'rating': np.size})
animeStatsNuevo.head(15)

,rating
anime_id,
1,12873
5,5446
6,8920
7,1977
8,259
15,1827
16,3007
17,476
18,1370


Ahora vamos a calcular el promedio de rating de cada una de los animes.

In [ ]:
animeStatsPromedio = ratings.groupby('anime_id').agg({'rating': np.mean})

animeStatsPromedio = animeStatsPromedio.sort_values('rating', ascending=False)
animeStatsPromedio.head(10)

C:\Users\dsegu\AppData\Local\Temp\ipykernel_18392\2981807696.py:1: FutureWarning: The provided callable <function mean at 0x000001B5B178CCC0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  animeStatsPromedo = ratings.groupby('anime_id').agg({'rating': np.mean})


,rating
anime_id,
28977,9.465500
820,9.436782
32281,9.430332
5114,9.325238
9969,9.295900
9253,9.267864
15417,9.251340
918,9.247951
11061,9.245528


El límite de 100 calificaciones quizás es muy bajo, pero ya nos sirve. Ahora unimos estos datos con el dataframe de película similares a StarWars.

In [34]:
# df = animeStatsNuevo[animeStatsNuevo].join(pd.DataFrame(similarAnimesToHatsukoi, columns=['similarity']))
# df.head()


Ahora mostrar ordenados por similaridad.

In [35]:
# df.sort_values(['similarity'], ascending=False).head(20)
#pd.set_option('display.max_rows', None)
#print(df)

#### Filtrado Colaborativo

In [36]:
corrMatrix = animeRatings.corr(method='pearson', min_periods=100)
corrMatrix.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,33241,33338,33372,33421,33524,33558,33569,33964,34103,34240
anime_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.524380,0.313323,0.123672,0.105945,0.085844,0.232479,0.116691,0.145075,0.246859,...,NaN,NaN,NaN,NaN,0.108911,0.124834,NaN,NaN,0.110706,0.068303
5,0.524380,1.000000,0.310094,0.220518,NaN,0.105251,0.185607,NaN,0.242504,0.145550,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.084029
6,0.313323,0.310094,1.000000,0.256794,0.346773,0.321294,0.190827,0.199182,0.201520,0.191706,...,NaN,NaN,NaN,NaN,NaN,0.247423,NaN,NaN,NaN,0.253530
7,0.123672,0.220518,0.256794,1.000000,NaN,0.486356,0.211432,NaN,0.242511,0.135147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.105945,NaN,0.346773,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Ahora vamos a producir recomendaciones para el usuario ID 0, añadido manualmente al dataset. Le gusta Hunter x Hunter, pero odia School dates

In [37]:
myRatings = pd.Series({11061: 10, 2476: 1})

Ahora vamos a recorrer cada anime calificado por el usuario y a crear una lista de posibles recomendaciones basadas en animes similares a los calificados.

Para cada anime calificada vamos a recuperar la lista de animes similares de la matriz de correlación. Luego se escala el puntaje de correlación según la calificación dada por el usuario en el anime que es similar, entonces los animes que le gustaron más van a contar más que las que no.

In [38]:
simCandidates = pd.Series(dtype='float64')

for anime_id, rating in myRatings.items():
    print(f"Añadiendo animes similares a {anime_id}...")
    
    # Recuperar los animes similares a los calificadas
    sims = corrMatrix[anime_id].dropna()
    
    # Escalar la similaridad multiplicando la correlación por la calificación del usuario
    sims = sims.map(lambda x: x * rating)
    
    # Añadir el puntaje a la lista de candidatos similares
    simCandidates = pd.concat([simCandidates, sims])

# Mirar los resultados
print("Ordenando...")
simCandidates = simCandidates.groupby(simCandidates.index).sum()  # Sumar puntuaciones duplicadas
simCandidates.sort_values(inplace=True, ascending=False)
filteredSims = simCandidates.drop(myRatings.index)
print(filteredSims.head(10))

Añadiendo animes similares a 11061...
Añadiendo animes similares a 2476...
Ordenando...
21635    5.167938
31174    4.805412
17389    4.559158
25313    4.434506
880      4.340234
19193    4.334015
6165     4.277379
24781    4.258107
6489     4.215640
136      4.201218
dtype: float64


Añadir los nombres de los animes para mostrar unos datos más visuales.

In [39]:
# Convertir la Series en DataFrame para hacer merge
simCandidates_df = filteredSims.reset_index()
simCandidates_df.columns = ['anime_id', 'score']

# Merge con el DataFrame de animes para obtener los nombres
simCandidates_df = simCandidates_df.merge(animes, on='anime_id')

# Reordenar columnas para legibilidad
simCandidates_df = simCandidates_df[['anime_id', 'name', 'score']]

# Mostrar las 10 recomendaciones principales con nombres
simCandidates_df.head(10)


,anime_id,name,score
0,21635,GJ-bu@,5.167938
1,31174,Osomatsu-san,4.805412
2,17389,Kingdom 2nd Season,4.559158
3,25313,Gintama: Jump Festa 2014 Special,4.434506
4,880,Aa! Megami-sama!: Sorezore no Tsubasa,4.340234
5,19193,Ghost in the Shell: Arise - Border:3 Ghost Tears,4.334015
6,6165,White Album 2nd Season,4.277379
7,24781,Imawa no Kuni no Alice (OVA),4.258107
8,6489,Zero no Tsukaima: Princesses no Rondo Picture ...,4.215640
9,136,Hunter x Hunter,4.201218
